# Capstone project Assignment: Week 3

In [33]:
#import libraries
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_rows', 500)

#### I use *beautifulsoup* to scrape data from the given wikipedia page:https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

right_table=soup.find('table', class_='wikitable sortable')
right_table.head()


NameError: name 'urllib' is not defined

Loop through "right_table" which is a compilation of the table in html format. Find text enclosed within 'tr' and 'td' and save in separate lists using findAll.

In [22]:
A=[]
B=[]
C=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
  



Convert to a pandas dataframe. This is the raw version of the table which still needs to be cleaned.

In [43]:
df_toronto=pd.DataFrame(A,columns=['PostalCode'])
df_toronto['Borough']=B
df_toronto['Neighbourhood']=C

df_t=df_toronto.replace('\n','', regex=True)
df_t.head(10)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [39]:
dropval = df_t[ df_t['Borough'] == 'Not assigned' ].index
df_t.drop(dropval , inplace=True)
df_t.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Check if there are any neighbourhoods having value "Not assigned"

In [49]:
exist = 'Not assigned' in df_t["Neighbourhood"]
exist

False

In [50]:
df_t.shape

(180, 3)